## **Cálculo del transformador**

In [8]:
import math
import numpy as np

le = 124e-3
u0 = math.pi * 4e-7
ur = 2000
Ae= 353e-6
Amin = 345e-6
f = 50000
T = 1/f
D = 0.7
Vo = 300
Vin = 12
Bsat = 400e-3 # Peor caso datasheet.
J = 4e6
kw = 0.4
Aw = 375.55e-6

min_diam_cable = 0.25e-3

Re = le/(u0*ur*Amin)
print(Re)
Al = 1/Re

V_N1 = Vin/2
V_N2 = Vo / D


### Del notebook del inductor
I1 = 3.25

ratio_N1_N2 = V_N1/V_N2
print("N1/N2: " + str(ratio_N1_N2))
print("N2/N1: " + str(1/ratio_N1_N2))


143008.78944489147
N1/N2: 0.013999999999999999
N2/N1: 71.42857142857143


In [9]:
### N1 minimo por Bsat
N1_min = 1/2 * Vin/2 * T * D/2 / (Amin * Bsat * 2)
N1_min = math.ceil(N1_min)
print("N1 min: " + str(N1_min))

N1 min: 1


In [10]:
######### Chequeo N1 min #############

#### Minima corriente inductor filtro.
Lf = 0.0072503 # H
Ilf_med = 0.13  # A
delta_Ilf = Vo * (1-D) * T/2 / Lf
Ilf_min = Ilf_med - delta_Ilf/2
print("Ilf_min: " + str(Ilf_min))

#### I mag secundario
def calc_Imag2(N1):
  return Vin / 2 * T * D/2 * ratio_N1_N2 / (N1_min**2 * Al)

Imag_2_initial = calc_Imag2(N1_min)
print("Imag_2_initial: " + str(Imag_2_initial))

Imag_2 = Imag_2_initial
while (Imag_2 > Ilf_min):
  N1_min+=1
  Imag_2 = calc_Imag2(N1_min)
  
print("Imag_2: " + str(Imag_2))
print("\nN1_min: " + str(N1_min))




Ilf_min: 0.06793360274747251
Imag_2_initial: 0.0840891681935962
Imag_2: 0.02102229204839905

N1_min: 2


Ahora vamos a revisar si con ese numero de vueltas el ancho del cable nos alcanza.

In [11]:
skin = 66e-3/math.sqrt(f)

if (min_diam_cable < 2*skin):
  print("Se desprecia el efecto skin.")

#### Limite por J.

N1_max = (J * Aw * kw) / (I1 * 2)

N2_max = (J * Aw * kw) / (I1 * ratio_N1_N2 * 2)

#### Limite por area.

Scu2 = I1 * ratio_N1_N2 / J
diam_Scu = math.sqrt(Scu2 * 4 / math.pi) #Diametro cobre

Scu_min_diam = (min_diam_cable ** 2) * math.pi / 4
N_max_scu2 = math.floor(Aw * 0.7 * kw / Scu_min_diam) 

N2_max_final = N_max_scu2 if N_max_scu2 < N2_max else N2_max

N1_max_final = N2_max_final * ratio_N1_N2

print("Vueltas primario max: " + str(math.floor(N1_max_final)))
print("Vueltas secundario max: " + str(N2_max_final))

Se desprecia el efecto skin.
Vueltas primario max: 29
Vueltas secundario max: 2142


In [12]:
### Se fija N1 < N1_max_final
N1 = 2100

In [13]:
L1 = (N1*ratio_N1_N2)**2 / Re
L2 = (N1) **2 / Re

Calculo del capacitor de la Buck

In [14]:
C = delta_Ilf / (8*0.05*Vo*f)

print("El valor del capacitor es: " + str(C))
print("L1 " + str(L1))
print("L2 " + str(L2))

print("L2/L1: " + str(int(L2/L1)))
print("N2/N1: " + str(1/ratio_N1_N2))

El valor del capacitor es: 2.068879908417583e-08
L1 0.006044104025739492
L2 30.83726543744639
L2/L1: 5102
N2/N1: 71.42857142857143
